# Réplication Zhang (2021)
## *Pairs trading with general state space models*

---

**Tables répliquées:**
- **Table 2**: Coefficients estimés des modèles I et II
- **Table 3**: Performance (Return, Std Dev, Sharpe, Calmar, Pain Index) pour chaque stratégie (A, B, C)
- **Tables A1-A6**: Annexes Banking pairs (Model I + Strategy A vs Model II + Strategy C)

**Paramètres:**
- Transaction costs: **20 bps**
- Risk-free rate: **2%**

** Note importante:** Pour l'Out-of-Sample, **pas de look-ahead bias**:
- `n_std` optimisé uniquement sur In-Sample
- Moyenne `C` et écart-type `σ` calculés uniquement sur In-Sample

In [1]:
from __future__ import annotations
import sys
from pathlib import Path
import itertools
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from dataclasses import dataclass
from typing import Tuple, List, Dict
import warnings
warnings.filterwarnings('ignore')

PROJECT_ROOT = Path('').resolve().parent
DATA_FILE = PROJECT_ROOT / "data" / "dataGQ.xlsx"
TRANSACTION_COST_BP = 20.0
RISK_FREE_RATE = 0.02

print(f" Data: {DATA_FILE}")
print(f" Costs: {TRANSACTION_COST_BP} bps | Risk-free: {RISK_FREE_RATE*100}%")

try:
    from numba import njit
    NUMBA_AVAILABLE = True
    print(" Numba enabled")
except ImportError:
    NUMBA_AVAILABLE = False
    def njit(*args, **kwargs):
        def decorator(func): return func
        return decorator if not (len(args) == 1 and callable(args[0])) else args[0]

📊 Data: C:\Users\Marius\PycharmProjects\Gestion-Quantitative\pairs-ssm\data\dataGQ.xlsx
💰 Costs: 20.0 bps | Risk-free: 2.0%
✅ Numba enabled


In [2]:
# Universes & Dates
LARGE_BANKS = ['JPM', 'BAC', 'WFC', 'C', 'USB']
SMALL_BANKS = ['CPF', 'BANC', 'CUBI', 'NBHC', 'FCF']
MAIN_PAIRS = [('PEP', 'KO'), ('EWT', 'EWH')]

FULL_SAMPLE_START, FULL_SAMPLE_END = '2012-01-03', '2019-06-28'
EWT_EWH_END = '2019-05-01'
IN_SAMPLE_START, IN_SAMPLE_END = '2012-01-10', '2018-01-01'
OUT_SAMPLE_START, OUT_SAMPLE_END = '2018-01-01', '2019-12-01'

In [3]:
@dataclass
class PairData:
    PA: pd.Series
    PB: pd.Series
    asset_a: str
    asset_b: str
    @property
    def n_obs(self): return len(self.PA)

@dataclass
class ModelParams:
    theta0: float = 0.0
    theta1: float = 0.95
    q_base: float = 1e-4
    q_het: float = 0.0
    r: float = 1e-4
    @property
    def is_homoscedastic(self): return self.q_het < 1e-10

def load_pair_data(filepath, col_a, col_b, start_date, end_date):
    df = pd.read_excel(filepath)
    if col_a in df.columns:
        if 'Date' in df.columns: df = df.set_index('Date')
        elif 'Unnamed: 0' in df.columns: df = df.set_index('Unnamed: 0')
        df.index = pd.to_datetime(df.index)
        PA, PB = df[col_a].dropna(), df[col_b].dropna()
    else:
        col_a_bb = f'{col_a} US Equity' if f'{col_a} US Equity' in df.columns else f'{col_a} US Equity '
        col_b_bb = f'{col_b} US Equity' if f'{col_b} US Equity' in df.columns else f'{col_b} US Equity '
        def get_series(df, col):
            col_idx = df.columns.get_loc(col)
            return pd.DataFrame({'date': pd.to_datetime(df.iloc[:, col_idx-1], errors='coerce'),
                                 'price': pd.to_numeric(df[col], errors='coerce')}
                               ).dropna().drop_duplicates('date').set_index('date').sort_index()['price']
        PA, PB = get_series(df, col_a_bb), get_series(df, col_b_bb)
    common = PA.index.intersection(PB.index)
    PA, PB = PA.loc[common], PB.loc[common]
    mask = (PA.index >= pd.to_datetime(start_date)) & (PA.index <= pd.to_datetime(end_date))
    return PairData(PA.loc[mask], PB.loc[mask], col_a, col_b)

In [4]:
# Kalman Filters
@njit(cache=True)
def halton_sequence_njit(size, base):
    seq = np.zeros(size)
    for i in range(size):
        n, f, r = i+1, 1.0, 0.0
        while n > 0: f /= base; r += f * (n % base); n //= base
        seq[i] = r
    return seq

@njit(cache=True)
def kalman_filter_njit(y, theta0, theta1, q, r):
    n = len(y)
    x = theta0/(1-theta1) if abs(theta1) < 0.999 else y[0]
    P = q/(1-theta1**2) if abs(theta1) < 0.999 else q*10
    x_filt, loglik = np.zeros(n), 0.0
    for t in range(n):
        if t > 0: x = theta0 + theta1*x; P = theta1**2*P + q
        v, S = y[t] - x, P + r
        if S > 1e-12:
            K = P/S; x += K*v; P = (1-K)*P
            loglik += -0.5*(np.log(2*np.pi) + np.log(S) + v*v/S)
        x_filt[t] = x
    return loglik, x_filt

@njit(cache=True)
def qmckf_njit(y, theta0, theta1, q_base, q_het, r, n_particles):
    n = len(y)
    x, P = y[0], q_base + q_het*y[0]**2
    x_filt, loglik = np.zeros(n), 0.0
    h1, h2 = halton_sequence_njit(n_particles, 2), halton_sequence_njit(n_particles, 3)
    for i in range(n_particles): h1[i] = max(1e-10, min(1-1e-10, h1[i])); h2[i] = max(1e-10, min(1-1e-10, h2[i]))
    z = np.sqrt(-2*np.log(h1)) * np.cos(2*np.pi*h2)
    samples, f_samples = np.zeros(n_particles), np.zeros(n_particles)
    for t in range(n):
        if t == 0: x_p, P_p = x, P
        else:
            sqrt_P = np.sqrt(max(P, 1e-12))
            for i in range(n_particles): samples[i] = x + sqrt_P*z[i]; f_samples[i] = theta0 + theta1*samples[i]
            x_p = np.mean(f_samples)
            P_p = np.var(f_samples) + np.mean(q_base + q_het*samples**2)
        v, S = y[t] - x_p, P_p + r
        if S > 1e-12:
            K = P_p/S; x = x_p + K*v; P = (1-K)*P_p
            loglik += -0.5*(np.log(2*np.pi) + np.log(S) + v*v/S)
        else: x, P = x_p, P_p
        x_filt[t] = x
    return loglik, x_filt

In [5]:
# Strategies
@njit(cache=True)
def strategy_A_njit(x, U, L, C):
    n, sig, pos = len(x), np.zeros(len(x)), 0
    for t in range(n):
        if pos == 0:
            if x[t] >= U[t]: pos = -1
            elif x[t] <= L[t]: pos = 1
        elif pos == 1 and x[t] >= C: pos = 0
        elif pos == -1 and x[t] <= C: pos = 0
        sig[t] = pos
    return sig

@njit(cache=True)
def strategy_B_njit(x, U, L):
    n, sig, pos = len(x), np.zeros(len(x)), 0
    for t in range(1, n):
        if x[t-1] < U[t-1] and x[t] >= U[t]: pos = -1
        elif x[t-1] > L[t-1] and x[t] <= L[t]: pos = 1
        sig[t] = pos
    return sig

@njit(cache=True)
def strategy_C_njit(x, U, L, C):
    n, sig, pos = len(x), np.zeros(len(x)), 0
    for t in range(1, n):
        prev, curr = x[t-1], x[t]
        U_p, U_c, L_p, L_c = U[t-1], U[t], L[t-1], L[t]
        entry_s = (prev > U_p) and (curr <= U_c)
        entry_l = (prev < L_p) and (curr >= L_c)
        exit_l = (prev < C) and (curr >= C)
        exit_s = (prev > C) and (curr <= C)
        stop_s = (prev < U_p) and (curr >= U_c)
        stop_l = (prev > L_p) and (curr <= L_c)
        if pos == 0:
            if entry_s: pos = -1
            elif entry_l: pos = 1
        elif pos == 1 and (exit_l or stop_l): pos = 0
        elif pos == -1 and (exit_s or stop_s): pos = 0
        sig[t] = pos
    return sig

@njit(cache=True)
def compute_thresholds_njit(x_filt, q_base, q_het, n_std, is_hetero):
    n = len(x_filt)
    C, sigma = np.mean(x_filt), np.std(x_filt)
    U, L = np.zeros(n), np.zeros(n)
    if is_hetero and q_het > 1e-10:
        g_x = np.sqrt(q_base + q_het * x_filt**2)
        mean_g = np.mean(g_x)
        for t in range(n):
            s_t = g_x[t]/mean_g * sigma
            U[t], L[t] = C + n_std*s_t, C - n_std*s_t
    else:
        for t in range(n): U[t], L[t] = C + n_std*sigma, C - n_std*sigma
    return U, L, C

In [6]:
# Backtest with full metrics
@njit(cache=True)
def backtest_full_njit(signals, x_filt, cost_bp, rf):
    n = len(signals)
    pnl = np.zeros(n)
    n_trades = 0
    cost_factor = 2.0 * cost_bp / 10000.0
    for t in range(1, n):
        dx = x_filt[t] - x_filt[t-1]
        pos_change = abs(signals[t] - signals[t-1])
        if pos_change > 0: n_trades += 1
        pnl[t] = signals[t]*dx - pos_change*cost_factor
    cum_pnl = np.cumsum(pnl)
    ann_ret = cum_pnl[-1] / (n/252.0)
    ann_std = np.std(pnl) * np.sqrt(252.0)
    sharpe = (ann_ret - rf) / ann_std if ann_std > 1e-10 else 0.0
    # Max DD & Calmar
    peak = cum_pnl[0]
    max_dd = 0.0
    for t in range(n):
        if cum_pnl[t] > peak: peak = cum_pnl[t]
        dd = peak - cum_pnl[t]
        if dd > max_dd: max_dd = dd
    calmar = ann_ret / max_dd if max_dd > 1e-10 else 0.0
    # Pain Index
    peak = cum_pnl[0]
    total_dd = 0.0
    for t in range(n):
        if cum_pnl[t] > peak: peak = cum_pnl[t]
        total_dd += peak - cum_pnl[t]
    pain = total_dd / n
    return ann_ret, ann_std, sharpe, calmar, pain, n_trades

@njit(cache=True)
def grid_search_full_njit(x_filt, q_base, q_het, is_hetero, strategy, cost_bp, rf):
    best_n, best_sr = 1.0, -1e10
    best_ret, best_std, best_calmar, best_pain, best_trades = 0.0, 0.0, 0.0, 0.0, 0
    for i in range(25):
        n_std = 0.1 + i*0.1
        U, L, C = compute_thresholds_njit(x_filt, q_base, q_het, n_std, is_hetero)
        if strategy == 0: sig = strategy_A_njit(x_filt, U, L, C)
        elif strategy == 1: sig = strategy_B_njit(x_filt, U, L)
        else: sig = strategy_C_njit(x_filt, U, L, C)
        ret, std, sr, calmar, pain, trades = backtest_full_njit(sig, x_filt, cost_bp, rf)
        if trades > 0 and sr > best_sr:
            best_sr, best_n = sr, n_std
            best_ret, best_std, best_calmar, best_pain, best_trades = ret, std, calmar, pain, trades
    return best_n, best_ret, best_std, best_sr, best_calmar, best_pain, best_trades

In [7]:
# OOS without look-ahead bias
@njit(cache=True)
def get_is_statistics_njit(x_filt, q_base, q_het, is_hetero):
    C, sigma = np.mean(x_filt), np.std(x_filt)
    if is_hetero and q_het > 1e-10:
        mean_g = np.mean(np.sqrt(q_base + q_het * x_filt**2))
    else:
        mean_g = 1.0
    return C, sigma, mean_g

@njit(cache=True)
def backtest_with_is_params_njit(x_oos, q_base, q_het, is_hetero, strategy, 
                                  n_std_is, C_is, sigma_is, mean_g_is, cost_bp, rf):
    """OOS backtest using IS parameters - NO LOOK-AHEAD BIAS"""
    n = len(x_oos)
    U, L = np.zeros(n), np.zeros(n)
    if is_hetero and q_het > 1e-10:
        for t in range(n):
            g_t = np.sqrt(q_base + q_het * x_oos[t]**2)
            s_t = g_t / mean_g_is * sigma_is
            U[t], L[t] = C_is + n_std_is*s_t, C_is - n_std_is*s_t
    else:
        thr = n_std_is * sigma_is
        for t in range(n): U[t], L[t] = C_is + thr, C_is - thr
    if strategy == 0: sig = strategy_A_njit(x_oos, U, L, C_is)
    elif strategy == 1: sig = strategy_B_njit(x_oos, U, L)
    else: sig = strategy_C_njit(x_oos, U, L, C_is)
    pnl = np.zeros(n)
    cost_factor = 2.0 * cost_bp / 10000.0
    for t in range(1, n):
        pnl[t] = sig[t]*(x_oos[t]-x_oos[t-1]) - abs(sig[t]-sig[t-1])*cost_factor
    ann_ret = np.sum(pnl) / (n/252.0)
    ann_std = np.std(pnl) * np.sqrt(252.0)
    sharpe = (ann_ret - rf) / ann_std if ann_std > 1e-10 else 0.0
    return ann_ret, sharpe

In [8]:
# Estimation
def estimate_gamma_ols(log_PA, log_PB):
    X = np.column_stack([np.ones(len(log_PB)), log_PB])
    return float(np.linalg.lstsq(X, log_PA, rcond=None)[0][1])

def estimate_model_I(y):
    y_mean, y_var = np.mean(y), np.var(y)
    rho = np.corrcoef(y[:-1]-y_mean, y[1:]-y_mean)[0,1]
    t1_init = float(np.clip(rho, 0.8, 0.99))
    z0 = [y_mean*(1-t1_init), np.arctanh(t1_init), np.log(y_var*(1-t1_init**2)*0.7+1e-10), np.log(y_var*0.3+1e-10)]
    def neg_ll(z):
        try: ll, _ = kalman_filter_njit(y, z[0], np.tanh(z[1]), np.exp(z[2]), np.exp(z[3])); return -ll if np.isfinite(ll) else 1e10
        except: return 1e10
    res = minimize(neg_ll, z0, method='L-BFGS-B', bounds=[(-0.5,0.5), (np.arctanh(0.5),np.arctanh(0.999)), (np.log(1e-8),np.log(1)), (np.log(1e-8),np.log(1))])
    p = ModelParams(theta0=res.x[0], theta1=np.tanh(res.x[1]), q_base=np.exp(res.x[2]), r=np.exp(res.x[3]))
    ll, x_filt = kalman_filter_njit(y, p.theta0, p.theta1, p.q_base, p.r)
    return p, x_filt, ll

def estimate_model_II(y):
    y_mean = np.mean(y)
    best_ll, best_p, best_f = -np.inf, None, None
    for t0, t1, qb, qh, r in [(y_mean*0.01,0.95,5e-4,0.1,0.01), (y_mean*0.01,0.93,3e-4,0.13,0.011), (y_mean*0.01,0.96,1e-3,0.08,0.008)]:
        z0 = [t0, np.arctanh(t1), np.log(qb), np.log(qh), np.log(r)]
        def neg_ll(z):
            try: ll, _ = qmckf_njit(y, z[0], np.tanh(z[1]), np.exp(z[2]), np.exp(z[3]), np.exp(z[4]), 50); return -ll if np.isfinite(ll) else 1e10
            except: return 1e10
        try:
            res = minimize(neg_ll, z0, method='L-BFGS-B', bounds=[(-0.1,0.1), (np.arctanh(0.85),np.arctanh(0.99)), (np.log(1e-6),np.log(5e-3)), (np.log(0.05),np.log(0.3)), (np.log(5e-3),np.log(0.05))], options={'maxiter':500})
            p = ModelParams(theta0=res.x[0], theta1=np.tanh(res.x[1]), q_base=np.exp(res.x[2]), q_het=np.exp(res.x[3]), r=np.exp(res.x[4]))
            ll, xf = qmckf_njit(y, p.theta0, p.theta1, p.q_base, p.q_het, p.r, 100)
            if ll > best_ll: best_ll, best_p, best_f = ll, p, xf
        except: continue
    if best_p is None:
        best_p = ModelParams(theta0=0, theta1=0.95, q_base=3e-4, q_het=0.1, r=0.01)
        best_ll, best_f = qmckf_njit(y, 0, 0.95, 3e-4, 0.1, 0.01, 100)
    return best_p, best_f, best_ll

In [9]:
# Warm-up JIT
if NUMBA_AVAILABLE:
    print(" JIT...")
    d = np.random.randn(100)
    _ = kalman_filter_njit(d, 0, 0.95, 0.001, 0.001)
    _ = qmckf_njit(d, 0, 0.95, 0.001, 0.1, 0.01, 50)
    U, L, C = compute_thresholds_njit(d, 0.001, 0.1, 1.0, True)
    _ = strategy_A_njit(d, U, L, C); _ = strategy_B_njit(d, U, L); _ = strategy_C_njit(d, U, L, C)
    _ = backtest_full_njit(np.zeros(100), d, 20, 0.02)
    _ = get_is_statistics_njit(d, 0.001, 0.1, True)
    _ = backtest_with_is_params_njit(d, 0.001, 0.1, True, 2, 1.0, 0, 0.01, 1.0, 20, 0.02)
    print(" Done!")

⏳ JIT...
✅ Done!


---
# TABLE 2: Coefficients des Modèles

In [10]:
print(" TABLE 2: Model Coefficients")
results_t2 = []
for col_a, col_b in MAIN_PAIRS:
    end = EWT_EWH_END if col_a == 'EWT' else FULL_SAMPLE_END
    try:
        pair = load_pair_data(DATA_FILE, col_a, col_b, FULL_SAMPLE_START, end)
        log_PA, log_PB = np.log(pair.PA.values), np.log(pair.PB.values)
        gamma = estimate_gamma_ols(log_PA, log_PB)
        y = log_PA - gamma * log_PB
        p1, _, ll1 = estimate_model_I(y)
        p2, _, ll2 = estimate_model_II(y)
        results_t2.append({'Pair': f'{col_a}-{col_b}', 'γ': gamma,
            'θ₀(I)': p1.theta0, 'θ₁(I)': p1.theta1, 'σ²η(I)': p1.q_base, 'σ²ε(I)': p1.r, 'LL(I)': ll1,
            'θ₀(II)': p2.theta0, 'θ₁(II)': p2.theta1, 'θ₂(II)': p2.q_base, 'θ₃(II)': p2.q_het, 'σ²ε(II)': p2.r, 'LL(II)': ll2})
        print(f" {col_a}-{col_b}")
    except Exception as e: print(f" {col_a}-{col_b}: {e}")
df_t2 = pd.DataFrame(results_t2)
display(df_t2.round(6))

📊 TABLE 2: Model Coefficients
✅ PEP-KO
✅ EWT-EWH


,Pair,γ,θ₀(I),θ₁(I),σ²η(I),σ²ε(I),LL(I),θ₀(II),θ₁(II),θ₂(II),θ₃(II),σ²ε(II),LL(II)
0,PEP-KO,1.996393,-0.051169,0.982428,0.000197,0.0,5363.952605,-0.100000,0.964055,0.000001,0.05,0.005,-944.786111
1,EWT-EWH,1.006229,0.006231,0.981633,0.000077,0.0,6112.644764,0.045487,0.850000,0.000001,0.05,0.005,2291.575456


---
# TABLE 3: Performance par Stratégie (A, B, C)

In [11]:
print(" TABLE 3: Performance by Strategy")
results_t3 = []
for col_a, col_b in MAIN_PAIRS:
    end = EWT_EWH_END if col_a == 'EWT' else FULL_SAMPLE_END
    try:
        pair = load_pair_data(DATA_FILE, col_a, col_b, FULL_SAMPLE_START, end)
        log_PA, log_PB = np.log(pair.PA.values), np.log(pair.PB.values)
        gamma = estimate_gamma_ols(log_PA, log_PB)
        y = log_PA - gamma * log_PB
        p1, f1, _ = estimate_model_I(y)
        p2, f2, _ = estimate_model_II(y)
        for model, params, xf in [('Model I', p1, f1), ('Model II', p2, f2)]:
            is_het = not params.is_homoscedastic
            qh = params.q_het if is_het else 0.0
            for si, sn in [(0,'A'), (1,'B'), (2,'C')]:
                _, ret, std, sr, cal, pain, _ = grid_search_full_njit(xf, params.q_base, qh, is_het, si, TRANSACTION_COST_BP, RISK_FREE_RATE)
                results_t3.append({'Pair': f'{col_a}-{col_b}', 'Model': model, 'Strategy': sn,
                    'Return': ret, 'Std Dev': std, 'Sharpe': sr, 'Calmar': cal, 'Pain Index': pain})
        print(f" {col_a}-{col_b}")
    except Exception as e: print(f" {col_a}-{col_b}: {e}")

df_t3 = pd.DataFrame(results_t3)
for pair in df_t3['Pair'].unique():
    print(f"\n{'='*90}\n  {pair}\n{'='*90}")
    print(f"{'Model':<12}{'Strategy':>8}{'Return':>10}{'Std Dev':>10}{'Sharpe':>10}{'Calmar':>10}{'Pain Idx':>10}")
    print("-"*90)
    for _, r in df_t3[df_t3['Pair']==pair].iterrows():
        print(f"{r['Model']:<12}{r['Strategy']:>8}{r['Return']:>10.4f}{r['Std Dev']:>10.4f}{r['Sharpe']:>10.4f}{r['Calmar']:>10.4f}{r['Pain Index']:>10.4f}")

📊 TABLE 3: Performance by Strategy
✅ PEP-KO
✅ EWT-EWH

  PEP-KO
Model       Strategy    Return   Std Dev    Sharpe    Calmar  Pain Idx
------------------------------------------------------------------------------------------
Model I            A    0.0456    0.1189    0.2152    0.2272    0.0202
Model I            B    0.1916    0.2240    0.7662    0.9548    0.0499
Model I            C    0.1846    0.1310    1.2563    2.0252    0.0159
Model II           A    0.0372    0.0616    0.2795    0.5831    0.0089
Model II           B    0.1897    0.2216    0.7658    0.9484    0.0484
Model II           C    0.1812    0.1259    1.2807    1.9161    0.0137

  EWT-EWH
Model       Strategy    Return   Std Dev    Sharpe    Calmar  Pain Idx
------------------------------------------------------------------------------------------
Model I            A    0.0521    0.0873    0.3677    0.4195    0.0127
Model I            B    0.0936    0.1399    0.5258    0.6233    0.0396
Model I            C    0.0974   

---
# ANNEXES A1-A6: Banking Pairs
##  OOS uses IS parameters (n_std, C, σ)

In [12]:
def analyze_pair_appendix(pair, cost_bp=20.0, rf=0.02):
    log_PA, log_PB = np.log(pair.PA.values), np.log(pair.PB.values)
    gamma = estimate_gamma_ols(log_PA, log_PB)
    y = log_PA - gamma * log_PB
    p1, f1, _ = estimate_model_I(y)
    _, ret1, _, sr1, _, _, _ = grid_search_full_njit(f1, p1.q_base, 0, False, 0, cost_bp, rf)
    p2, f2, _ = estimate_model_II(y)
    _, ret2, _, sr2, _, _, _ = grid_search_full_njit(f2, p2.q_base, p2.q_het, True, 2, cost_bp, rf)
    return {'Stock #1': pair.asset_a, 'Stock #2': pair.asset_b,
            'M1_Return': ret1, 'M1_Sharpe': sr1, 'M2_Return': ret2, 'M2_Sharpe': sr2,
            'Imp_Return': (ret2/ret1-1)*100 if abs(ret1)>1e-6 else 0, 'Imp_Sharpe': (sr2/sr1-1)*100 if abs(sr1)>1e-6 else 0}

def analyze_pair_is_oos(filepath, col_a, col_b, is_start, is_end, oos_start, oos_end, cost_bp=20.0, rf=0.02):
    """IS/OOS analysis WITHOUT look-ahead bias"""
    # IS
    pair_is = load_pair_data(filepath, col_a, col_b, is_start, is_end)
    log_PA_is, log_PB_is = np.log(pair_is.PA.values), np.log(pair_is.PB.values)
    gamma = estimate_gamma_ols(log_PA_is, log_PB_is)
    y_is = log_PA_is - gamma * log_PB_is
    
    p1, f1_is, _ = estimate_model_I(y_is)
    n_std_m1, ret1_is, _, sr1_is, _, _, _ = grid_search_full_njit(f1_is, p1.q_base, 0, False, 0, cost_bp, rf)
    C1_is, sig1_is, mg1_is = get_is_statistics_njit(f1_is, p1.q_base, 0, False)
    
    p2, f2_is, _ = estimate_model_II(y_is)
    n_std_m2, ret2_is, _, sr2_is, _, _, _ = grid_search_full_njit(f2_is, p2.q_base, p2.q_het, True, 2, cost_bp, rf)
    C2_is, sig2_is, mg2_is = get_is_statistics_njit(f2_is, p2.q_base, p2.q_het, True)
    
    r_is = {'Stock #1': col_a, 'Stock #2': col_b, 'M1_Return': ret1_is, 'M1_Sharpe': sr1_is,
            'M2_Return': ret2_is, 'M2_Sharpe': sr2_is,
            'Imp_Return': (ret2_is/ret1_is-1)*100 if abs(ret1_is)>1e-6 else 0,
            'Imp_Sharpe': (sr2_is/sr1_is-1)*100 if abs(sr1_is)>1e-6 else 0}
    
    # OOS with IS params
    pair_oos = load_pair_data(filepath, col_a, col_b, oos_start, oos_end)
    log_PA_oos, log_PB_oos = np.log(pair_oos.PA.values), np.log(pair_oos.PB.values)
    y_oos = log_PA_oos - gamma * log_PB_oos
    
    _, f1_oos = kalman_filter_njit(y_oos, p1.theta0, p1.theta1, p1.q_base, p1.r)
    _, f2_oos = qmckf_njit(y_oos, p2.theta0, p2.theta1, p2.q_base, p2.q_het, p2.r, 100)
    
    ret1_oos, sr1_oos = backtest_with_is_params_njit(f1_oos, p1.q_base, 0, False, 0, n_std_m1, C1_is, sig1_is, mg1_is, cost_bp, rf)
    ret2_oos, sr2_oos = backtest_with_is_params_njit(f2_oos, p2.q_base, p2.q_het, True, 2, n_std_m2, C2_is, sig2_is, mg2_is, cost_bp, rf)
    
    r_oos = {'Stock #1': col_a, 'Stock #2': col_b, 'M1_Return': ret1_oos, 'M1_Sharpe': sr1_oos,
             'M2_Return': ret2_oos, 'M2_Sharpe': sr2_oos,
             'Imp_Return': (ret2_oos/ret1_oos-1)*100 if abs(ret1_oos)>1e-6 else 0,
             'Imp_Sharpe': (sr2_oos/sr1_oos-1)*100 if abs(sr1_oos)>1e-6 else 0}
    return r_is, r_oos

def display_table(df, title, page=0, note=""):
    print(f"\n{'_'*95}")
    if page: print(f"{' '*70}G. Zhang{page:>7}")
    print(f"\n{title}\n{'_'*95}")
    print(f"{' '*44}Model I + Strategy A    Model II + Strategy C    Improvement (%)")
    print(f"{'Pair':<8}{'Stock #1':<12}{'Stock #2':<12}{'Return':>10}{'Sharpe':>10}{'Return':>10}{'Sharpe':>10}{'Return':>10}{'Sharpe':>10}")
    print('_'*95)
    for i, (_, r) in enumerate(df.iterrows(), 1):
        print(f"{i:<8}{r['Stock #1']:<12}{r['Stock #2']:<12}{r['M1_Return']:>10.4f}{r['M1_Sharpe']:>10.4f}{r['M2_Return']:>10.4f}{r['M2_Sharpe']:>10.4f}{r['Imp_Return']:>10.2f}{r['Imp_Sharpe']:>10.2f}")
    print('_'*95)
    for stat, fn in [('Mean', 'mean'), ('Min', 'min'), ('Max', 'max'), ('Median', 'median')]:
        print(f"{stat:<32}{getattr(df['M1_Return'], fn)():>10.4f}{getattr(df['M1_Sharpe'], fn)():>10.4f}{getattr(df['M2_Return'], fn)():>10.4f}{getattr(df['M2_Sharpe'], fn)():>10.4f}{getattr(df['Imp_Return'], fn)():>10.2f}{getattr(df['Imp_Sharpe'], fn)():>10.2f}")
    if note: print(f"\nNote: {note}")

In [13]:
# Table A1: Large Banks
print("📊 Table A1: Large Banks")
large_pairs = list(itertools.combinations(LARGE_BANKS, 2))
res_large = []
for s1, s2 in large_pairs:
    try:
        r = analyze_pair_appendix(load_pair_data(DATA_FILE, s1, s2, FULL_SAMPLE_START, FULL_SAMPLE_END), TRANSACTION_COST_BP, RISK_FREE_RATE)
        res_large.append(r); print(f"   {s1}-{s2}")
    except Exception as e: print(f"   {s1}-{s2}: {e}")
df_a1_large = pd.DataFrame(res_large)
if not df_a1_large.empty: display_table(df_a1_large, "Table A1 - Panel A: Pairs of Large Banks", 1582)

📊 Table A1: Large Banks
  ✅ JPM-BAC
  ✅ JPM-WFC
  ✅ JPM-C
  ✅ JPM-USB
  ✅ BAC-WFC
  ✅ BAC-C
  ✅ BAC-USB
  ✅ WFC-C
  ✅ WFC-USB
  ✅ C-USB

_______________________________________________________________________________________________
                                                                      G. Zhang   1582

Table A1 - Panel A: Pairs of Large Banks
_______________________________________________________________________________________________
                                            Model I + Strategy A    Model II + Strategy C    Improvement (%)
Pair    Stock #1    Stock #2        Return    Sharpe    Return    Sharpe    Return    Sharpe
_______________________________________________________________________________________________
1       JPM         BAC             0.0823    0.6002    0.0985    1.0097     19.62     68.24
2       JPM         WFC             0.0112   -0.0829    0.0795    0.7803    609.90  -1041.20
3       JPM         C               0.0869    0.7500    0.0

In [14]:
# Table A1: Small Banks
print("📊 Table A1: Small Banks")
small_pairs = list(itertools.combinations(SMALL_BANKS, 2))
res_small = []
for s1, s2 in small_pairs:
    try:
        r = analyze_pair_appendix(load_pair_data(DATA_FILE, s1, s2, FULL_SAMPLE_START, FULL_SAMPLE_END), TRANSACTION_COST_BP, RISK_FREE_RATE)
        res_small.append(r); print(f"   {s1}-{s2}")
    except Exception as e: print(f"   {s1}-{s2}: {e}")
df_a1_small = pd.DataFrame(res_small)
if not df_a1_small.empty: display_table(df_a1_small, "Table A1 - Panel B: Pairs of Small Banks", 1582)

📊 Table A1: Small Banks
  ✅ CPF-BANC
  ✅ CPF-CUBI
  ✅ CPF-NBHC
  ✅ CPF-FCF
  ✅ BANC-CUBI
  ✅ BANC-NBHC
  ✅ BANC-FCF
  ✅ CUBI-NBHC
  ✅ CUBI-FCF
  ✅ NBHC-FCF

_______________________________________________________________________________________________
                                                                      G. Zhang   1582

Table A1 - Panel B: Pairs of Small Banks
_______________________________________________________________________________________________
                                            Model I + Strategy A    Model II + Strategy C    Improvement (%)
Pair    Stock #1    Stock #2        Return    Sharpe    Return    Sharpe    Return    Sharpe
_______________________________________________________________________________________________
1       CPF         BANC            0.0858    0.4346    0.1217    1.4414     41.82    231.69
2       CPF         CUBI            0.0581    0.2455    0.1761    1.5663    203.24    538.01
3       CPF         NBHC            0.1

In [15]:
# Table A2: Large × Small
print("📊 Table A2: Large × Small Banks")
cross_pairs = list(itertools.product(LARGE_BANKS, SMALL_BANKS))
res_cross = []
for s1, s2 in cross_pairs:
    try:
        r = analyze_pair_appendix(load_pair_data(DATA_FILE, s1, s2, FULL_SAMPLE_START, FULL_SAMPLE_END), TRANSACTION_COST_BP, RISK_FREE_RATE)
        res_cross.append(r); print(f"   {s1}-{s2}")
    except Exception as e: print(f"   {s1}-{s2}: {e}")
df_a2 = pd.DataFrame(res_cross)
if not df_a2.empty: display_table(df_a2, "Table A2: Pairs Between Large and Small Banks", 1583)

📊 Table A2: Large × Small Banks
  ✅ JPM-CPF
  ✅ JPM-BANC
  ✅ JPM-CUBI
  ✅ JPM-NBHC
  ✅ JPM-FCF
  ✅ BAC-CPF
  ✅ BAC-BANC
  ✅ BAC-CUBI
  ✅ BAC-NBHC
  ✅ BAC-FCF
  ✅ WFC-CPF
  ✅ WFC-BANC
  ✅ WFC-CUBI
  ✅ WFC-NBHC
  ✅ WFC-FCF
  ✅ C-CPF
  ✅ C-BANC
  ✅ C-CUBI
  ✅ C-NBHC
  ✅ C-FCF
  ✅ USB-CPF
  ✅ USB-BANC
  ✅ USB-CUBI
  ✅ USB-NBHC
  ✅ USB-FCF

_______________________________________________________________________________________________
                                                                      G. Zhang   1583

Table A2: Pairs Between Large and Small Banks
_______________________________________________________________________________________________
                                            Model I + Strategy A    Model II + Strategy C    Improvement (%)
Pair    Stock #1    Stock #2        Return    Sharpe    Return    Sharpe    Return    Sharpe
_______________________________________________________________________________________________
1       JPM         CPF             0.0

In [16]:
# Table A3: IS/OOS Large Banks
print("📊 Table A3: IS/OOS Large Banks (NO LOOK-AHEAD BIAS)")
res_a3_is, res_a3_oos = [], []
for s1, s2 in large_pairs:
    try:
        r_is, r_oos = analyze_pair_is_oos(DATA_FILE, s1, s2, IN_SAMPLE_START, IN_SAMPLE_END, OUT_SAMPLE_START, OUT_SAMPLE_END, TRANSACTION_COST_BP, RISK_FREE_RATE)
        res_a3_is.append(r_is); res_a3_oos.append(r_oos); print(f"   {s1}-{s2}")
    except Exception as e: print(f"   {s1}-{s2}: {e}")
df_a3_is, df_a3_oos = pd.DataFrame(res_a3_is), pd.DataFrame(res_a3_oos)
if not df_a3_is.empty: display_table(df_a3_is, "Table A3 - Panel A: In Sample (Large Banks)", 1584, f"Data: {IN_SAMPLE_START} to {IN_SAMPLE_END}")
if not df_a3_oos.empty: display_table(df_a3_oos, "Table A3 - Panel B: Out of Sample (Large Banks)", 1584, f"OOS uses IS params. Data: {OUT_SAMPLE_START} to {OUT_SAMPLE_END}")

📊 Table A3: IS/OOS Large Banks (NO LOOK-AHEAD BIAS)
  ✅ JPM-BAC
  ✅ JPM-WFC
  ✅ JPM-C
  ✅ JPM-USB
  ✅ BAC-WFC
  ✅ BAC-C
  ✅ BAC-USB
  ✅ WFC-C
  ✅ WFC-USB
  ✅ C-USB

_______________________________________________________________________________________________
                                                                      G. Zhang   1584

Table A3 - Panel A: In Sample (Large Banks)
_______________________________________________________________________________________________
                                            Model I + Strategy A    Model II + Strategy C    Improvement (%)
Pair    Stock #1    Stock #2        Return    Sharpe    Return    Sharpe    Return    Sharpe
_______________________________________________________________________________________________
1       JPM         BAC             0.0556    0.4277    0.0506    0.7943     -8.99     85.73
2       JPM         WFC             0.0371    0.2841    0.0834    0.9286    124.68    226.86
3       JPM         C       

In [17]:
# Table A4: IS/OOS Small Banks
print(" Table A4: IS/OOS Small Banks")
res_a4_is, res_a4_oos = [], []
for s1, s2 in small_pairs:
    try:
        r_is, r_oos = analyze_pair_is_oos(DATA_FILE, s1, s2, IN_SAMPLE_START, IN_SAMPLE_END, OUT_SAMPLE_START, OUT_SAMPLE_END, TRANSACTION_COST_BP, RISK_FREE_RATE)
        res_a4_is.append(r_is); res_a4_oos.append(r_oos); print(f"   {s1}-{s2}")
    except Exception as e: print(f"   {s1}-{s2}: {e}")
df_a4_is, df_a4_oos = pd.DataFrame(res_a4_is), pd.DataFrame(res_a4_oos)
if not df_a4_is.empty: display_table(df_a4_is, "Table A4 - Panel A: In Sample (Small Banks)", 1585)
if not df_a4_oos.empty: display_table(df_a4_oos, "Table A4 - Panel B: Out of Sample (Small Banks)", 1585)

📊 Table A4: IS/OOS Small Banks
  ✅ CPF-BANC
  ✅ CPF-CUBI
  ✅ CPF-NBHC
  ✅ CPF-FCF
  ✅ BANC-CUBI
  ✅ BANC-NBHC
  ✅ BANC-FCF
  ✅ CUBI-NBHC
  ✅ CUBI-FCF
  ✅ NBHC-FCF

_______________________________________________________________________________________________
                                                                      G. Zhang   1585

Table A4 - Panel A: In Sample (Small Banks)
_______________________________________________________________________________________________
                                            Model I + Strategy A    Model II + Strategy C    Improvement (%)
Pair    Stock #1    Stock #2        Return    Sharpe    Return    Sharpe    Return    Sharpe
_______________________________________________________________________________________________
1       CPF         BANC            0.1632    0.9392    0.1541    1.2735     -5.58     35.59
2       CPF         CUBI            0.0756    0.3427    0.2171    1.7639    187.34    414.75
3       CPF         NBHC     

In [18]:
# Tables A5 & A6: IS/OOS Large × Small
print(" Tables A5 & A6: IS/OOS Large × Small Banks")
res_a5, res_a6 = [], []
for s1, s2 in cross_pairs:
    try:
        r_is, r_oos = analyze_pair_is_oos(DATA_FILE, s1, s2, IN_SAMPLE_START, IN_SAMPLE_END, OUT_SAMPLE_START, OUT_SAMPLE_END, TRANSACTION_COST_BP, RISK_FREE_RATE)
        res_a5.append(r_is); res_a6.append(r_oos); print(f"   {s1}-{s2}")
    except Exception as e: print(f"   {s1}-{s2}: {e}")
df_a5, df_a6 = pd.DataFrame(res_a5), pd.DataFrame(res_a6)
if not df_a5.empty: display_table(df_a5, "Table A5: In Sample (Large × Small Banks)", 1586, f"Data: {IN_SAMPLE_START} to {IN_SAMPLE_END}")
if not df_a6.empty: display_table(df_a6, "Table A6: Out of Sample (Large × Small Banks)", 1587, f"OOS uses IS params. Data: {OUT_SAMPLE_START} to {OUT_SAMPLE_END}")

📊 Tables A5 & A6: IS/OOS Large × Small Banks
  ✅ JPM-CPF
  ✅ JPM-BANC
  ✅ JPM-CUBI
  ✅ JPM-NBHC
  ✅ JPM-FCF
  ✅ BAC-CPF
  ✅ BAC-BANC
  ✅ BAC-CUBI
  ✅ BAC-NBHC
  ✅ BAC-FCF
  ✅ WFC-CPF
  ✅ WFC-BANC
  ✅ WFC-CUBI
  ✅ WFC-NBHC
  ✅ WFC-FCF
  ✅ C-CPF
  ✅ C-BANC
  ✅ C-CUBI
  ✅ C-NBHC
  ✅ C-FCF
  ✅ USB-CPF
  ✅ USB-BANC
  ✅ USB-CUBI
  ✅ USB-NBHC
  ✅ USB-FCF

_______________________________________________________________________________________________
                                                                      G. Zhang   1586

Table A5: In Sample (Large × Small Banks)
_______________________________________________________________________________________________
                                            Model I + Strategy A    Model II + Strategy C    Improvement (%)
Pair    Stock #1    Stock #2        Return    Sharpe    Return    Sharpe    Return    Sharpe
_______________________________________________________________________________________________
1       JPM         CPF       

In [19]:
print("\n" + "="*90)
print(" REPLICATION COMPLETE")
print("="*90)
print(f"Transaction costs: {TRANSACTION_COST_BP} bps")
print(f"Risk-free rate: {RISK_FREE_RATE*100}%")
print(f"OOS: NO look-ahead bias (n_std, C, σ from IS only)")


✅ REPLICATION COMPLETE
Transaction costs: 20.0 bps
Risk-free rate: 2.0%
OOS: NO look-ahead bias (n_std, C, σ from IS only)
